This notebook is to manually check the computation of particular parts of the optimization.

In [1]:
import numpy as np
import pickle
from scipy import stats

# Objective function of $w$ optimization

The objective function has the form
\begin{equation}
    -w^T y + \psi_\alpha \lVert w \rVert_2 - \lambda_k^T K^T w + \frac{\mu}{2} \lVert K^T w \rVert_2^2 - \mu (h + A^T c_k)^T K^T w,
\end{equation}
which to start is equivalent to
\begin{equation}
    -w^T y + \psi_\alpha \lVert w \rVert_2 + \frac{\mu}{2} \lVert K^T w \rVert_2^2 - \mu h^T K^T w,
\end{equation}
since $\lambda_k = c_k = 0$.

As such, I need the following items
1. $w$ --> generated by Gaussian with known random seed
2. $y$ --> affine corrected observation
3. $\psi_\alpha$ --> on Cheyenne
4. $\mu$ --> fixed up front
5. $K^T w$ --> saved in the hash table
6. $h$ --> functional

In [2]:
# affine corrected observation
with open('../data/y_affine_corrected.npy', 'rb') as f:
    y = np.load(f)
m = y.shape[0]
print(y[:10])

[-0.43160245  1.55670918 -0.19545318  1.94694523  1.20822325 -0.39844148
 -1.67594377  0.46664634  0.93883433  2.17580318]


In [3]:
# Gaussian vector
np.random.seed(12345)
w_sp = stats.norm().rvs(m)
print(w_sp[:10])

# make hash value for the above
w_key = hash(w_sp.tobytes())
print(w_key)

[-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057  1.39340583
  0.09290788  0.28174615  0.76902257  1.24643474]
4910459656328879700


In [4]:
# slack factor
with open('../data/opt_res_cont.pkl', 'rb') as f:
    opt_slack = pickle.load(f)
PSI2 = stats.chi2.ppf(q=.95, df=1) + opt_slack[1]
psi_alpha = np.sqrt(PSI2)
print(psi_alpha)

145.0614723275884


In [5]:
# table of K^T evals
with open('../data/h_table_lep.pkl', 'rb') as f:
    KTw_dict = pickle.load(f)

In [6]:
# functional
with open('../data/na_june_functional.npy', 'rb') as f:
    h = np.load(f)

In [32]:
# isolate the keys
dict_keys = list(KTw_dict.keys())
print(dict_keys)

[0, -8025139033159738307, -7350210967711714390, 5107400654505320952, -6967323695701719286]


In [24]:
w_diff = w_sp - KTw_dict[-8025139033159738307]['w']
w_diff[w_diff > 0]

array([2.22044605e-16, 3.46944695e-18, 2.22044605e-16, 5.55111512e-17])

In [29]:
def obj_func(w, y, psi, mu, KTw, h):
    t1 = -np.dot(w, y)
    t2 = psi * np.linalg.norm(w)
    t3 = (mu / 2) * np.linalg.norm(KTw)
    t4 = - mu * np.dot(h, KTw)
    return t1 + t2 + t3 + t4

In [34]:
# objective function at the start (idx 1)
KEY_IDX = 1
MU = 1e3
obj_func(
    w=KTw_dict[dict_keys[KEY_IDX]]['w'],
    y=y, psi=psi_alpha, mu=MU,
    KTw=KTw_dict[dict_keys[KEY_IDX]]['KTw'], h=h
)

23980.801163379634

In [41]:
# objective function at the start (idx 2)
KEY_IDX = 2
MU = 1e3
obj_func(
    w=KTw_dict[dict_keys[KEY_IDX]]['w'],
    y=y, psi=psi_alpha, mu=MU,
    KTw=KTw_dict[dict_keys[KEY_IDX]]['KTw'], h=h
)

24069.33131309384

In [36]:
# objective function at the start (idx 3)
KEY_IDX = 3
MU = 1e3
obj_func(
    w=KTw_dict[dict_keys[KEY_IDX]]['w'],
    y=y, psi=psi_alpha, mu=MU,
    KTw=KTw_dict[dict_keys[KEY_IDX]]['KTw'], h=h
)

23997.31694915535

In [37]:
# objective function at the start (idx 4)
KEY_IDX = 4
MU = 1e3
obj_func(
    w=KTw_dict[dict_keys[KEY_IDX]]['w'],
    y=y, psi=psi_alpha, mu=MU,
    KTw=KTw_dict[dict_keys[KEY_IDX]]['KTw'], h=h
)

23983.8923887543